In [ ]:
import numpy as np, analysis as ana, plottools as pl, prjlib
from matplotlib.pyplot import *
from scipy.optimize import curve_fit

In [ ]:
def likefunc(A,c0,c1,mscb,icov,bi=0):
    bn = np.shape(mscb)
    return np.exp(ana.lnLHL(A/c0*np.ones(bn),c1*mscb,icov,bi))

def fitlikefunc(xdat,ydat,yerr,mscb,icov,c1=None,bi=0):
    bn = np.shape(mscb)
    index = np.where(yerr==0.)
    x = np.delete(xdat,index)
    y = np.delete(ydat,index)
    s = np.delete(yerr,index)
    if c1 is None:
        like = lambda x, c0, c1: np.exp(ana.lnLHLs(x/c0,c1*mscb,icov,bi))
        p, __ = curve_fit(like,x,y,sigma=s,p0=[1.,1.])
        return p[0], p[1]
    else:
        like = lambda x, c0: np.exp(ana.lnLHLs(x/c0,mscb,icov,bi))
        p, __ = curve_fit(like,x,y,sigma=s,p0=[1.])
        return p[0]

In [ ]:
def loaddata(bn,lmin,Acb): #output binned spectra
    ps, __ = prjlib.filename_init(rlmin='200',stype='lcmb',PSA='s14&15_deep56',doreal='True',dearot='True')
    if Acb=='0.0':
        pa = ps
    else:
        pa, __ = prjlib.filename_init(rlmin='200',stype='a'+str(Acb).replace('.','p'),PSA='s14&15_deep56')
    pA, __ = prjlib.filename_init(rlmin='200',stype='a0p3',PSA='s14&15_deep56')
    mb, mb0, mb1 = prjlib.binning_all(bn,lmin=lmin,Lsp=2048)
    sn0b = prjlib.binned_cl(ps.quad.f['EB'].n0bs,mb0,mb1)
    scb = prjlib.binned_cl_rlz(ps.quad.f['EB'].cl,1,200,mb0,mb1) + sn0b[None,:]
    acb = prjlib.binned_cl_rlz(pa.quad.f['EB'].cl,1,100,mb0,mb1) + sn0b[None,:]
    Acb = prjlib.binned_cl_rlz(pA.quad.f['EB'].cl,1,100,mb0,mb1) + sn0b[None,:]
    ocb = prjlib.binned_cl(ps.quad.f['EB'].cl[0],mb0,mb1) + sn0b
    macb = np.mean(acb,axis=0)
    mscb = np.mean(scb[:100],axis=0)
    dacb = (np.mean(Acb,axis=0) - mscb)/3.
    icov = ana.get_cov(scb[100:],cinv=True)
    return mb, ocb, scb, mscb, acb, macb, dacb, icov

In [ ]:
def fit_c0c1(Ab,mscb,icov,histbn=None,lmin=20,do_plot=False):
    # output c0 and c1
    nv = np.shape(Ab)[0]
    bn = np.shape(Ab)[1]
    c0 = np.zeros(bn)
    c1 = np.zeros(bn)
    if histbn is None:
        histbn = np.int(np.sqrt(nv))
    for bi in range(bn):
        N, be = np.histogram(Ab[:,bi],bins=histbn)
        An = (be[1:]+be[:-1])*.5
        if bi<=4: # well fitted by assuming c1=1
            c0[bi] = fitlikefunc(An,N/np.max(N),np.sqrt(N)/np.max(N),mscb,icov,c1=1.,bi=bi)
            c1[bi] = 1.
        if bi>=5: # need c1 correction
            c0[bi], c1[bi] = fitlikefunc(An,N/np.max(N),np.sqrt(N)/np.max(N),mscb,icov,bi=bi) # fit likelihood

        if do_plot:
            pl.hist_errorbars(Ab[:,bi],fsize=[4,3],bins=histbn,ymax=1.2)
            A = np.arange(np.min(Ab[:,bi]),np.max(Ab[:,bi]),(np.max(Ab[:,bi])-np.min(Ab[:,bi]))/50.)
            Lh = np.array([likefunc(a,c0[bi],c1[bi],mscb,icov,bi) for a in A])
            plot(A,Lh/np.max(Lh),'r-',label='HL,c0='+str(c0[bi])[:4]+',c1='+str(c1[bi])[:4])
            legend(loc=0,frameon=False)
            #savefig('fig_fit_bin'+str(bi)+'.png')
            show()
            clf()

    return c0,c1

In [ ]:
As = np.array([0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.3])
c0 = np.zeros((len(As),10))
c1 = np.zeros((len(As),10))
for i, Acb in enumerate(As):
    # load data
    mb, ocb, scb, mscb, acb, macb, dacb, icov =loaddata(10,lmin=20,Acb=str(Acb))
    # fit c0 and c1
    if Acb == 0.0:
        Ab = scb[:100,:]/mscb[None,:]
    else:
        Ab = acb[:100,:]/macb[None,:]
    c0[i,:], c1[i,:] = fit_c0c1(Ab,mscb,icov,lmin=20,do_plot=True)
    print(c0[i,0],c1[i,0])

In [ ]:
for bi in [0]:
    plot(As,c0[:,bi])
    plot(As,c1[:,bi])

In [ ]:
ai = 10
A = np.arange(0.,0.5,0.001)
Ai = np.zeros(100)
CL = np.zeros(100)
ylim(0,1)
xlim(0,0.25)
grid(True)
axhline(0.95,color='k',ls='--')
for i in range(100):
    Lh = prjlib.posterior(A,scb[i,:],mscb,dacb,icov,c0[ai,:],c1[ai,:])
    if i==0: Lh = prjlib.posterior(A,ocb,mscb,dacb,icov,c0[ai,:],c1[ai,:])
    #plot(A,Lh/np.max(Lh),'k--',alpha=.3)
    if i==0: plot(A,Lh/np.max(Lh),'k-')
    CDF = np.cumsum(Lh)/np.sum(Lh)
    #plot(A,CDF,'m-',alpha=.3)
    if i==0: plot(A,CDF,'r-',lw=2)
    Ai[i] = A[np.argmax(Lh)]
    CL[i] = A[np.abs(CDF-0.95).argmin()]

print(CL[0],np.median(CL),np.mean(CL))
print(np.sqrt(CL[0]*1e-4)*4*np.pi)
#savefig('fig_pdf.png')
show()
clf()